In [1]:
# !pip install -q ipython-autotime
%load_ext autoreload
%load_ext autotime
%autoreload 2

time: 16 ms (started: 2023-08-30 12:04:46 +03:00)


In [2]:
import requests, os, tarfile
from tqdm import tqdm
from data_download import download, extract_tgz

dataset_url = r'https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz'
segmentations = r'https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102segmentations.tgz'
chi_distance = r'https://www.robots.ox.ac.uk/~vgg/data/flowers/102/distancematrices102.mat'
labels = r'https://www.robots.ox.ac.uk/~vgg/data/flowers/102/imagelabels.mat'
splits = r'https://www.robots.ox.ac.uk/~vgg/data/flowers/102/setid.mat'

OVERWRITE = False

download(dataset_url, dataset_url.split('/')[-1], force_redownload=OVERWRITE)
# download(segmentations, segmentations.split('/')[-1], force_redownload=OVERWRITE)
# download(chi_distance, chi_distance.split('/')[-1], force_redownload=OVERWRITE)
download(labels, labels.split('/')[-1], force_redownload=OVERWRITE)
download(splits, splits.split('/')[-1], force_redownload=OVERWRITE)
extract_tgz(r'102flowers.tgz')

File already exists
File already exists
File already exists
File already extracted
time: 110 ms (started: 2023-08-30 12:04:46 +03:00)


In [3]:
import torch
import os
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from scipy.io import loadmat
import numpy as np
import pandas as pd
from data_processing import prepare_df, FlowerDataset


split_path = r"data/setid.mat"
labels_Path = r"data/imagelabels.mat"
data_root = r"data/102flowers/jpg"
train_split, test_split, val_split = prepare_df(split_path, labels_Path, data_root)

transformsations = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]
)

# train_dataset = FlowerDataset(train_split, transform=transformsations)
# test_dataset = FlowerDataset(test_split, transform=transformsations)
# val_dataset = FlowerDataset(val_split, transform=transformsations)


# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
# train_split

time: 1.27 s (started: 2023-08-30 12:04:46 +03:00)


In [4]:
import torchvision
import torchsummary
import torch.nn as nn
import segmentation_models_pytorch as smp



time: 1.12 s (started: 2023-08-30 12:04:47 +03:00)


In [5]:
def accuray_fn(ypred, y_true):
    """
    Calculate the accuracy of the prediction
    """
    return (ypred == y_true).sum() / len(ypred)


time: 0 ns (started: 2023-08-30 12:04:49 +03:00)


In [10]:
class Resnet50Flower102(nn.Module):
    def __init__(self, pretrained=True):
        super().__init__()
        self.model = torchvision.models.resnet50(pretrained=pretrained)
        self.model.fc = nn.Sequential(
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 102),
        )

    def forward(self, x):
        return self.model(x)



time: 0 ns (started: 2023-08-30 12:06:20 +03:00)


In [12]:
import torch
# from models  import Resnet50Flower102
from tqdm import tqdm
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = 'cpu'
model = Resnet50Flower102(pretrained=True).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
# optimizer = torch.optim.Adam(model.model.fc.parameters(), lr=3e-4)
train_dataset = FlowerDataset(train_split, transform=transformsations)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

time: 407 ms (started: 2023-08-30 12:07:21 +03:00)


In [9]:
total_loss = 0
for x, y in tqdm(train_loader, total=len(train_loader), desc="Training", leave=False):
    optimizer.zero_grad()
    x = x.to(device)
    y = y.to(device)
    y_hat = model(x)
    loss = loss_fn(y_hat, y)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
print(f"Loss: {total_loss / len(train_loader)}")
print(f"Accuracy: {accuray_fn(y_hat.argmax(1), y)}")

RuntimeError: cuDNN error: CUDNN_STATUS_EXECUTION_FAILED
You can try to repro this exception using the following code snippet. If that doesn't trigger the error, please include your original repro script when reporting this issue.

import torch
torch.backends.cuda.matmul.allow_tf32 = False
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.allow_tf32 = True
data = torch.randn([4, 1024, 14, 14], dtype=torch.float, device='cuda', requires_grad=True)
net = torch.nn.Conv2d(1024, 256, kernel_size=[1, 1], padding=[0, 0], stride=[1, 1], dilation=[1, 1], groups=1)
net = net.cuda().float()
out = net(data)
out.backward(torch.randn_like(out))
torch.cuda.synchronize()

ConvolutionParams 
    memory_format = Contiguous
    data_type = CUDNN_DATA_FLOAT
    padding = [0, 0, 0]
    stride = [1, 1, 0]
    dilation = [1, 1, 0]
    groups = 1
    deterministic = false
    allow_tf32 = true
input: TensorDescriptor 00000179106BD3D0
    type = CUDNN_DATA_F

time: 6.83 s (started: 2023-08-30 12:05:20 +03:00)


In [ ]:
# for Epoch in range(20):
#     total_loss = 0
#     accuracy = 0
#     for x, y in tqdm(train_loader, total=len(train_loader), desc="Training", leave=False):
#         x, y = next(iter(train_loader))
#         x = x.to(device)
#         y = y.to(device).type(torch.FloatTensor).requires_grad_(True)
#         optimizer.zero_grad()
#         y_hat = model(x).argmax(dim=1).type(torch.FloatTensor).requires_grad_(True)
#         loss = loss_fn( y_hat, y)
#         accuray = (y_hat == y).sum()/len(y)
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
#     print(y, y_hat)
#     print(total_loss/len(train_loader))
#     print(accuray)

tensor([96., 94.,  2., 29., 74., 87.,  8., 60., 87.,  5., 96., 67., 29., 51.,
        66., 50.], requires_grad=True) tensor([ 4., 67., 74., 26., 71., 68., 45.,  4., 24.,  4.,  4., 54., 74., 69.,
         4., 92.], requires_grad=True)
42660.555509334416
tensor(0.)


KeyboardInterrupt: 

time: 2min 15s (started: 2023-08-30 11:54:16 +03:00)
